In [1]:
import matplotlib.pyplot as plt
import anndata as adata
import scanpy as sc
#from wmb import cemba, mm10
from ALLCools.mcds import MCDS
from ALLCools.clustering import significant_pc_test
from ALLCools.plot import *
import pandas as pd
#from harmonypy import run_harmony

In [2]:
adata_sc = adata.read_h5ad("/data2st1/junyi/output/sc_canghao/6_combined_normalized_anndata_with_celltype.L1.h5ad")

In [14]:
adata_sc.layers['normalized'] = adata_sc.X

In [32]:
for sample in adata_sc.obs['batch_name'].unique():
    adata_sample = adata_sc[adata_sc.obs['batch_name'] == sample]
    sc.pp.calculate_qc_metrics(adata_sample, qc_vars=["mt"], inplace=True)
    adata_sample.var.to_csv(f"/data2st1/junyi/pseudobulk/expression_bulk_{sample}.csv",index=False)


/home/junyichen/anaconda3/envs/allcools/lib/python3.8/site-packages/scanpy/preprocessing/_qc.py:135: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[obs_metrics.columns] = obs_metrics
/home/junyichen/anaconda3/envs/allcools/lib/python3.8/site-packages/scanpy/preprocessing/_qc.py:135: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[obs_metrics.columns] = obs_metrics
/home/junyichen/anaconda3/envs/allcools/lib/python3.8/site-packages/scanpy/preprocessing/_qc.py:135: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[obs_metrics.columns] = obs_metrics
/home/junyichen/anaconda3/envs/allcools/lib/python3.8/site-packages/scanpy/preprocessing/_qc.py:135: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[obs_metrics.columns] = obs_metrics
/hom

In [30]:
adata_sample.var

,gene_ids,feature_types,genome,mt,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts
Xkr4,ENSMUSG00000051951,Gene Expression,mm10,False,9208,3.525880,1.509812,22.124493,41690,10.638041
Gm1992,ENSMUSG00000089699,Gene Expression,mm10,False,3520,0.432510,0.359428,70.230041,5114,8.539933
Gm19938,ENSMUSG00000102331,Gene Expression,mm10,False,1601,0.156631,0.145511,86.459743,1852,7.524561
Gm37381,ENSMUSG00000102343,Gene Expression,mm10,False,1,0.000085,0.000085,99.991543,1,0.693147
Rp1,ENSMUSG00000025900,Gene Expression,mm10,False,27,0.002453,0.002450,99.771651,29,3.401197
...,...,...,...,...,...,...,...,...,...,...
AC124606.1,ENSMUSG00000095523,Gene Expression,mm10,False,0,0.000000,0.000000,100.000000,0,0.000000
AC133095.2,ENSMUSG00000095475,Gene Expression,mm10,False,0,0.000000,0.000000,100.000000,0,0.000000
AC133095.1,ENSMUSG00000094855,Gene Expression,mm10,False,0,0.000000,0.000000,100.000000,0,0.000000
AC234645.1,ENSMUSG00000095019,Gene Expression,mm10,False,0,0.000000,0.000000,100.000000,0,0.000000


In [23]:
adata_sample.X.sum(axis=0).shape

(1, 32285)

In [3]:
adata_annoated = adata.read_h5ad("/data7/mark/STG/dataset/ST_HD_out/annotation.h5ad")

In [34]:
adata_annoated.obs

,batch,doublet_score,predicted_doublet,region,treatment,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,...,nCount_RNA,nFeature_RNA,RNA_snn_res.1,seurat_clusters,celltype.L1,Neurotransmitter,Neurotransmitter_celltype,celltype.L2,umap_1,umap_2
AAACCCAAGAAGCTCG-1-W25-01_Cortex,W25-01_Cortex,0.011502,False,Cortex,depression,664,664,884,4,0.452489,...,884,664,1,1,OPC-Oligo,NaN,NaN,MOL,-10.857223,3.677173
AAACCCAAGATGGTCG-1-W25-01_Cortex,W25-01_Cortex,0.007240,False,Cortex,depression,1082,1082,1839,3,0.163132,...,1839,1082,36,36,OPC-Oligo,NaN,NaN,MFOL,-12.893482,-0.641165
AAACCCAAGCAAATGT-1-W25-01_Cortex,W25-01_Cortex,0.006845,False,Cortex,depression,634,634,786,0,0.000000,...,786,634,8,8,Immune,NaN,NaN,Microglia,-3.188760,-15.681813
AAACCCAAGCGCCATC-1-W25-01_Cortex,W25-01_Cortex,0.033808,False,Cortex,depression,3704,3704,12251,0,0.000000,...,12251,3704,4,4,Neuron,NaN,Glutamatergic,L2/3 IT,7.971831,8.834544
AAACCCAAGCTCACTA-1-W25-01_Cortex,W25-01_Cortex,0.027597,False,Cortex,depression,803,803,1228,1,0.081433,...,1228,803,1,1,OPC-Oligo,NaN,NaN,MOL,-12.953283,1.713125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCATTCTTG-1-W26-02_PrefrontalCortex,W26-02_PrefrontalCortex,0.009447,False,PrefrontalCortex,control,921,921,1475,0,0.000000,...,1475,921,2,2,Astro-Epen,NaN,NaN,Astrocyte-Gfaplow,3.656259,-13.148583
TTTGTTGTCCTCACCA-1-W26-02_PrefrontalCortex,W26-02_PrefrontalCortex,0.008981,False,PrefrontalCortex,control,1653,1653,3067,0,0.000000,...,3067,1653,20,20,Neuron,NaN,GABAergic,Sst GABA,3.909881,-5.692503
TTTGTTGTCGCCAATA-1-W26-02_PrefrontalCortex,W26-02_PrefrontalCortex,0.042982,False,PrefrontalCortex,control,2020,2020,4236,0,0.000000,...,4236,2020,4,4,Neuron,NaN,Glutamatergic,L2/3 IT,9.189919,8.503694
TTTGTTGTCGGTTGTA-1-W26-02_PrefrontalCortex,W26-02_PrefrontalCortex,0.035941,False,PrefrontalCortex,control,1877,1877,3602,0,0.000000,...,3602,1877,4,4,Neuron,NaN,Glutamatergic,L2/3 IT,9.504958,7.957729


In [74]:
df_w26obs = adata_sc.obs[(adata_sc.obs.tech == '10x') & (adata_sc.obs.group == 'W')]

In [75]:
df_w26obs['barcode'] = df_w26obs.index.str.split('-').str[0]

/tmp/ipykernel_586907/247355735.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w26obs['barcode'] = df_w26obs.index.str.split('-').str[0]


In [66]:
df_annoatted = adata_annoated.obs

In [68]:
df_annoatted = df_annoatted[(df_annoatted.treatment == 'control') & (df_annoatted.region == 'PrefrontalCortex')]

In [72]:
df_annoatted['barcode'] = df_annoatted.index.str.split('-').str[0]

/tmp/ipykernel_586907/562785184.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_annoatted['barcode'] = df_annoatted.index.str.split('-').str[0]


In [82]:
df_w26obs['rowname'] = df_w26obs.index

/tmp/ipykernel_586907/836393030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w26obs['rowname'] = df_w26obs.index


In [83]:
df_obsmerged = df_w26obs.merge(df_annoatted, on='barcode', how='outer')

In [87]:
df_obsmerged.to_csv('/data2st1/junyi/output/df_PFC_annotation.csv')

In [78]:
df_obsmerged

,sample,group,n_genes_by_counts_x,log1p_n_genes_by_counts,total_counts_x,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,...,nCount_RNA,nFeature_RNA,RNA_snn_res.1,seurat_clusters,celltype.L1,Neurotransmitter,Neurotransmitter_celltype,celltype.L2,umap_1,umap_2
0,W3,W,847,6.742881,1343.0,7.203405,29.560685,38.644825,51.824274,74.162323,...,1343.0,847.0,18.0,18.0,Neuron,NaN,GABAergic,Pvalb GABA,9.937620,-4.755986
1,W3,W,2367,7.769801,5585.0,8.628018,29.597135,37.887198,47.090421,61.772605,...,5585.0,2367.0,4.0,4.0,Neuron,NaN,Glutamatergic,L2/3 IT,8.696033,7.428587
2,W3,W,1975,7.588830,3909.0,8.271293,26.502942,34.177539,43.975441,60.629317,...,3909.0,1975.0,9.0,9.0,Neuron,NaN,Glutamatergic,L6 IT,5.140928,8.976215
3,W3,W,1625,7.393878,2899.0,7.972466,25.974474,33.563298,43.428769,61.193515,...,2899.0,1625.0,10.0,10.0,Neuron,NaN,Glutamatergic,L6 CT,-1.043103,6.853716
4,W3,W,1296,7.167809,2210.0,7.701200,27.556561,35.429864,45.701357,63.981900,...,2210.0,1296.0,7.0,7.0,Neuron,NaN,Glutamatergic,L4/5 IT,6.615864,4.612892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17950,W3,W,921,6.826545,1475.0,7.297091,31.525424,39.050847,51.118644,71.457627,...,1475.0,921.0,2.0,2.0,Astro-Epen,NaN,NaN,Astrocyte-Gfaplow,3.656259,-13.148583
17951,W3,W,1653,7.410952,3067.0,8.028781,30.974894,37.463319,46.136290,62.406260,...,3067.0,1653.0,20.0,20.0,Neuron,NaN,GABAergic,Sst GABA,3.909881,-5.692503
17952,W3,W,2020,7.611348,4236.0,8.351611,30.406043,37.606232,45.963173,61.614731,...,4236.0,2020.0,4.0,4.0,Neuron,NaN,Glutamatergic,L2/3 IT,9.189919,8.503694
17953,W3,W,1877,7.537963,3602.0,8.189522,27.679067,35.258190,44.641866,61.771238,...,3602.0,1877.0,4.0,4.0,Neuron,NaN,Glutamatergic,L2/3 IT,9.504958,7.957729


In [11]:
adata_counts = adata_sc.layers["counts"]

In [19]:
adata_sc.var

,mt,n_cells_by_counts-0,mean_counts-0,log1p_mean_counts-0,pct_dropout_by_counts-0,total_counts-0,log1p_total_counts-0,highly_variable-0,means-0,dispersions-0,...,highly_variable_rank-1,variances-1,variances_norm-1,highly_variable_nbatches-1,highly_variable,highly_variable_rank,means,variances,variances_norm,highly_variable_nbatches
Xkr4,False,40547,15.705115,2.815715,13.004205,731984.0,13.503515,False,15.705115,1.828880,...,NaN,24.259528,1.052374,0,False,NaN,11.067082,246.025292,1.100350,0
Gm37381,False,99,0.002188,0.002186,99.787590,102.0,4.634729,False,0.002188,0.885140,...,NaN,0.000092,0.995483,0,False,NaN,0.001325,0.001399,0.963477,0
Rp1,False,1384,0.031754,0.031260,97.030553,1480.0,7.300473,False,0.031754,0.584985,...,NaN,0.002478,0.992670,0,False,NaN,0.019655,0.022097,0.918056,0
Sox17,False,98,0.003733,0.003726,99.789736,174.0,5.164786,True,0.003733,2.791106,...,1135.0,0.000613,1.133911,1,True,861.0,0.002398,0.007038,1.847386,5
Gm37323,False,13,0.000279,0.000279,99.972108,13.0,2.639057,False,0.000279,-0.820078,...,NaN,0.000031,0.500068,0,False,NaN,0.000177,0.000177,0.651889,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC168977.1,False,36,0.000772,0.000772,99.922760,36.0,3.610918,False,0.000772,1.998929,...,NaN,0.000031,0.500068,0,False,NaN,0.000467,0.000467,0.790622,0
AC149090.1,False,33180,4.590049,1.720988,28.810505,213933.0,12.273423,False,4.590049,1.670890,...,NaN,1.900554,1.052991,0,False,1812.0,3.094765,26.906226,1.122198,2
CAAA01118383.1,False,3490,0.083248,0.079963,92.512015,3880.0,8.263848,False,0.083248,1.323510,...,NaN,0.016454,0.953380,0,False,NaN,0.055607,0.064281,0.895764,0
Vmn2r122,False,0,0.000000,0.000000,100.000000,0.0,0.000000,False,0.000000,NaN,...,NaN,0.000000,0.000000,0,False,NaN,0.000000,0.000000,0.000000,0


In [26]:
adata_wt = adata_sc[(adata_sc.obs.group=="W") & (adata_sc.obs.tech=="10x")].copy()

In [28]:
adata_wt.obs

,sample,group,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,...,pct_counts_mt,leiden,pred_dmb,pred_mwb,pred_mdg,pred_mih,pred_mpd,sampleID,tech,louvain
AAACCCAAGCTAGCCC-1-0-0-1,W3,W,847,6.742881,1343.0,7.203405,29.560685,38.644825,51.824274,74.162323,...,0.000000,10,Blood: Erythrocyte,052 Pvalb Gaba,GABA,Pvalb,GABA-Lhx6,W_W3,10x,12
AAACCCAAGCTGGAGT-1-0-0-1,W3,W,2367,7.769801,5585.0,8.628018,29.597135,37.887198,47.090421,61.772605,...,0.107431,1,Blood: Erythrocyte,007 L2/3 IT CTX Glut,CA3-Pyr,L2/3 IT CTX,Cck-Tox,W_W3,10x,0
AAACCCAAGGATACCG-1-0-0-1,W3,W,1975,7.588830,3909.0,8.271293,26.502942,34.177539,43.975441,60.629317,...,0.000000,12,Blood: Erythrocyte,005 L5 IT CTX Glut,CA3-Pyr,L4/5 IT CTX,Cck-Tox,W_W3,10x,4
AAACCCAAGGGTCACA-1-0-0-1,W3,W,1625,7.393878,2899.0,7.972466,25.974474,33.563298,43.428769,61.193515,...,0.000000,3,Blood: Erythrocyte,030 L6 CT CTX Glut,CA3-Pyr,L6 CT CTX,Cck-Tox,W_W3,10x,5
AAACCCAAGTGCTAGG-1-0-0-1,W3,W,1296,7.167809,2210.0,7.701200,27.556561,35.429864,45.701357,63.981900,...,0.000000,4,Blood: Erythrocyte,006 L4/5 IT CTX Glut,CA3-Pyr,L4/5 IT CTX,Cck-Tox,W_W3,10x,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCATTCTTG-1-0-0-1,W3,W,921,6.826545,1475.0,7.297091,31.525424,39.050847,51.118644,71.457627,...,0.000000,16,Blood: Erythrocyte,319 Astro-TE NN,Astro-adult,Astro,Astrocytes,W_W3,10x,14
TTTGTTGTCCTCACCA-1-0-0-1,W3,W,1653,7.410952,3067.0,8.028781,30.974894,37.463319,46.136290,62.406260,...,0.000000,14,Blood: Erythrocyte,053 Sst Gaba,GABA,Sst,GABA-Lhx6,W_W3,10x,15
TTTGTTGTCGCCAATA-1-0-0-1,W3,W,2020,7.611348,4236.0,8.351611,30.406043,37.606232,45.963173,61.614731,...,0.000000,1,Blood: Erythrocyte,007 L2/3 IT CTX Glut,CA3-Pyr,L2/3 IT CTX,Cck-Tox,W_W3,10x,0
TTTGTTGTCGGTTGTA-1-0-0-1,W3,W,1877,7.537963,3602.0,8.189522,27.679067,35.258190,44.641866,61.771238,...,0.000000,1,Blood: Erythrocyte,007 L2/3 IT CTX Glut,CA3-Pyr,L2/3 IT CTX,Cck-Tox,W_W3,10x,0


In [29]:
adata_wt.var['gane_name'] = adata_wt.var.index

In [20]:
adata_raw = sc.read_10x_mtx("/data1st1/seq_from_our_lab/NovaSeq_mmBrain_X101SC24104268-Z01/X101SC24104268-Z01-J001/seeksoul_output/W1/step3/filtered_feature_bc_matrix")

In [30]:
var_merged = adata_raw.var.merge(adata_wt.var,how='outer',left_index=True,right_index=True,indicator=True)
var_merged

,gene_ids,feature_types,mt,n_cells_by_counts-0,mean_counts-0,log1p_mean_counts-0,pct_dropout_by_counts-0,total_counts-0,log1p_total_counts-0,highly_variable-0,...,variances_norm-1,highly_variable_nbatches-1,highly_variable,highly_variable_rank,means,variances,variances_norm,highly_variable_nbatches,gane_name,_merge
0610006L08Rik,ENSMUSG00000108652,Gene Expression,False,488.0,0.010771,0.010713,98.952969,502.0,6.220590,False,...,1.199852,2.0,False,1289.5,0.006552,0.006938,0.981850,2.0,0610006L08Rik,both
0610009B22Rik,ENSMUSG00000007777,Gene Expression,False,3681.0,0.086165,0.082654,92.102214,4016.0,8.298290,False,...,0.951403,0.0,False,NaN,0.055316,0.061725,0.870921,0.0,0610009B22Rik,both
0610010F05Rik,ENSMUSG00000042208,Gene Expression,False,29100.0,1.691684,0.990167,37.564367,78846.0,11.275265,False,...,0.936489,0.0,False,NaN,1.169863,3.464708,0.941099,0.0,0610010F05Rik,both
0610010K14Rik,ENSMUSG00000020831,Gene Expression,False,2797.0,0.065182,0.063146,93.998884,3038.0,8.019284,False,...,0.963431,0.0,False,NaN,0.041632,0.046843,0.880443,0.0,0610010K14Rik,both
0610012G03Rik,ENSMUSG00000107002,Gene Expression,False,9999.0,0.278944,0.246034,78.546601,13001.0,9.472858,False,...,0.993750,0.0,False,NaN,0.188735,0.264457,0.948035,0.0,0610012G03Rik,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mt-Nd3,ENSMUSG00000064360,Gene Expression,True,1271.0,0.033771,0.033213,97.273000,1574.0,7.362010,True,...,0.999874,0.0,True,1005.0,0.025651,0.045999,1.134224,1.0,mt-Nd3,both
mt-Nd4,ENSMUSG00000064363,Gene Expression,True,8277.0,0.706810,0.534626,82.241246,32943.0,10.402564,True,...,1.138509,1.0,True,62.0,0.468612,40.777472,8.289056,5.0,mt-Nd4,both
mt-Nd4l,ENSMUSG00000065947,Gene Expression,True,1777.0,0.069323,0.067026,96.187350,3231.0,8.080856,True,...,1.030246,0.0,True,897.0,0.043008,0.317972,2.459168,3.0,mt-Nd4l,both
mt-Nd5,ENSMUSG00000064367,Gene Expression,True,3155.0,0.224961,0.202909,93.230776,10485.0,9.257796,True,...,1.045469,0.0,True,314.0,0.143454,5.049482,5.491102,4.0,mt-Nd5,both


In [31]:
adata_wt.var = var_merged.loc[adata_wt.var.index]

In [32]:
adata_wt.var.set_index('gene_ids',inplace=True)

In [39]:
adata_counts = adata_wt.layers["counts"]

In [54]:
adata_counts

<17955x27734 sparse matrix of type '<class 'numpy.float32'>'
	with 28500402 stored elements in Compressed Sparse Row format>

In [56]:
var_merged

,gene_ids,feature_types,mt,n_cells_by_counts-0,mean_counts-0,log1p_mean_counts-0,pct_dropout_by_counts-0,total_counts-0,log1p_total_counts-0,highly_variable-0,...,variances_norm-1,highly_variable_nbatches-1,highly_variable,highly_variable_rank,means,variances,variances_norm,highly_variable_nbatches,gane_name,_merge
0610006L08Rik,ENSMUSG00000108652,Gene Expression,False,488.0,0.010771,0.010713,98.952969,502.0,6.220590,False,...,1.199852,2.0,False,1289.5,0.006552,0.006938,0.981850,2.0,0610006L08Rik,both
0610009B22Rik,ENSMUSG00000007777,Gene Expression,False,3681.0,0.086165,0.082654,92.102214,4016.0,8.298290,False,...,0.951403,0.0,False,NaN,0.055316,0.061725,0.870921,0.0,0610009B22Rik,both
0610010F05Rik,ENSMUSG00000042208,Gene Expression,False,29100.0,1.691684,0.990167,37.564367,78846.0,11.275265,False,...,0.936489,0.0,False,NaN,1.169863,3.464708,0.941099,0.0,0610010F05Rik,both
0610010K14Rik,ENSMUSG00000020831,Gene Expression,False,2797.0,0.065182,0.063146,93.998884,3038.0,8.019284,False,...,0.963431,0.0,False,NaN,0.041632,0.046843,0.880443,0.0,0610010K14Rik,both
0610012G03Rik,ENSMUSG00000107002,Gene Expression,False,9999.0,0.278944,0.246034,78.546601,13001.0,9.472858,False,...,0.993750,0.0,False,NaN,0.188735,0.264457,0.948035,0.0,0610012G03Rik,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mt-Nd3,ENSMUSG00000064360,Gene Expression,True,1271.0,0.033771,0.033213,97.273000,1574.0,7.362010,True,...,0.999874,0.0,True,1005.0,0.025651,0.045999,1.134224,1.0,mt-Nd3,both
mt-Nd4,ENSMUSG00000064363,Gene Expression,True,8277.0,0.706810,0.534626,82.241246,32943.0,10.402564,True,...,1.138509,1.0,True,62.0,0.468612,40.777472,8.289056,5.0,mt-Nd4,both
mt-Nd4l,ENSMUSG00000065947,Gene Expression,True,1777.0,0.069323,0.067026,96.187350,3231.0,8.080856,True,...,1.030246,0.0,True,897.0,0.043008,0.317972,2.459168,3.0,mt-Nd4l,both
mt-Nd5,ENSMUSG00000064367,Gene Expression,True,3155.0,0.224961,0.202909,93.230776,10485.0,9.257796,True,...,1.045469,0.0,True,314.0,0.143454,5.049482,5.491102,4.0,mt-Nd5,both


In [61]:
adata_wt.var.counts = adata_counts.sum(axis=0).T

/tmp/ipykernel_1006073/1207726834.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  adata_wt.var.counts = adata_counts.sum(axis=0).T


In [77]:
df_counts = adata_wt.var[["total_counts-0",'gane_name']].reset_index()
df_counts.columns = ['gene_ids','total_counts','gene_name']
df_counts.to_csv("/data8/junyi/pseudobulk_methlylation/expression_bulk.csv",index=False)